In [5]:
import pandas as pd
from scipy.fft import fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt
import zipfile 
import polars as pl
from scipy.stats import entropy, kurtosis

# Funções

Nota: Essas duas funções foram criadas pois fiz a extração de características diretamente no arquivo .zip do dataset. Dá pra fazer isso de uma maneira mais direta e performática, coletando diretamente do arquivo .csv, só fiz por este caminho devido a limitações de espaço em disco. Futuramente pretendo fazer uma versão mais performática deste código, coletando os dados diretamente dos csvs.

Note: These two functions were created because I extracted the features directly from the dataset's .zip file. It can be done in a more straightforward and efficient way by retrieving the data directly from the .csv file. I only followed this approach due to disk space limitations. In the future, I intend to create a more optimized version of this code, collecting the data directly from the CSVs.

## Get data from .zip File

Nota: Essas duas funções foram criadas pois fiz a extração de características diretamente no arquivo .zip do dataset. Dá pra fazer isso de uma maneira mais direta e performática, coletando diretamente do arquivo .csv, só fiz por este caminho devido a limitações de espaço em disco. Futuramente pretendo fazer uma versão mais performática deste código, coletando os dados diretamente dos csvs.

Note: These two functions were created because I extracted the features directly from the dataset's .zip file. It can be done in a more straightforward and efficient way by retrieving the data directly from the .csv file. I only followed this approach due to disk space limitations. In the future, I intend to create a more optimized version of this code, collecting the data directly from the CSVs.

In [1]:
def GetSubFolders(ZipFileName):
    zip_file = zipfile.ZipFile(ZipFileName)
    folder_list = []
    parquet_filename_list = []
    #filelist = []
    for file in zip_file.infolist():
        if file.filename.endswith('.csv'):
            file_list = file.filename.split('/')
            prefix = '_'.join(file_list[0:len(file_list)-1]).replace('.','_')
            folder = '/'.join(file_list[0:len(file_list)-1])
            folder_list.append(folder)
            folder_list = list(set(folder_list))
            folder_list.sort()
    return folder_list

def ReadCsvFromZipFile(ZipFileName, FolderName, ColumnNames, CastConfig):    
    zip_file = zipfile.ZipFile(ZipFileName)
    datasets = []
    for file in zip_file.infolist():
        if file.filename.endswith('.csv') and file.filename.startswith(FolderName):
            datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))

    return datasets

## Tacometer Rotating Speed estimate


In [2]:
# TacometerRotatingSpeedEstimate

# This function aims to estimate the rotating speed from the tacometer data (Sensor1). 

def TacometerRotatingSpeedEstimate(fs, N, raw_dataset):
# Iterate 4 times
    Rf_candidates = []
    for i in np.arange(4):
        
        # Locate the most significant peak |St(k)| and its index (ka)
        max_dataset = raw_dataset.select(['index','Sensor1_Magnitude', 'Frequency']).filter(pl.col('index') == np.argmax(raw_dataset['Sensor1_Magnitude']))
        ka = max_dataset['index']

        #Calculate the frequency estimate
        ft = (ka*fs)/(N)

        # Save the estimate in the vector 
        Rf_candidates.append(ft[0])

        # Set to zero for (ka-3) <= k <= (ka + 3)
        raw_dataset = raw_dataset.with_columns(Sensor1_Magnitude = (pl.when(pl.col('index').is_between(ka[0]-3,ka[0]+3)).then(0).otherwise(pl.col('Sensor1_Magnitude'))))
        i = i+1
        
    return round(min(Rf_candidates), 6) # Return the final rotating-speed estimate

def GetMagnitudeHarmonics(Rf, dataset, sensor):
    magnitude = dataset.select(['Frequency', sensor]).filter(pl.col('Frequency').is_between(Rf - 0.1, Rf+0.1))
    # Get the first 3 harmonics
    for i in np.linspace(2,3,2):
        filter_data = dataset.select(['Frequency',sensor]).filter(pl.col('Frequency').is_between(i*Rf - 0.1, i*Rf+0.1))
        magnitude = pl.concat([magnitude, filter_data], how='vertical')

    magnitude = magnitude.rename({sensor: "HarmonicMagnitude"})
    return magnitude

## Converting data to the spectrum space using FFT

In [3]:
def CalculateFFT(SignalData, SamplingRate):


    # Parâmetros do sinal
    f_s = 50000 # Frequência de amostragem (100 kHz)
    signal = np.array(SignalData)     # Sinal de entrada

    # Calcular FFT
    magnitude = fft(signal)
    magnitude = magnitude/len(SignalData)
    magnitude = magnitude[:len(SignalData)//2]
    frequency = fftfreq(len(signal), 1/f_s)
    frequency = frequency[:len(SignalData)//2]
    return abs(magnitude), frequency


def GetSpectrumData(RawData, SamplingRate):
    spectrum_array = []
    spectrum = {}
    for sensor in RawData.iter_columns():
        magnitude, frequency = CalculateFFT(sensor, SamplingRate)
        spectrum = pl.DataFrame(
            {       
                    sensor.name + "_Magnitude": magnitude,
                    #"Frequency": frequency
            }
        )
        spectrum_array.append(spectrum)
    frequency = pl.DataFrame(
            {
                "Frequency": frequency
            }
    )
    spectrum_array.append(frequency)
    spectrum_data = pl.concat(spectrum_array, how = "horizontal")
    return spectrum_data

def PlotFFTData(spectrum_data):
    fig, axs = plt.subplots(8, 1, layout='constrained')
    for i, sensor in enumerate(spectrum_data.columns[0:8]):
        axs[i].plot(spectrum_data["Frequency"], spectrum_data[sensor])
        axs[i].set_title(sensor)


    # Hide x labels and tick labels for all but bottom plot.
    for ax in axs:
        ax.label_outer()

# Features array for each dataset
#FrequencyFeatures = []
# Number of datasets
#N_datasets = len(folder_datasets)

#Features array for all datasets in the selected folder
#FrequencyFeaturesArray = np.zeros((len(folder_datasets),22))

def GetFrequencyFeaturesArray(folder_datasets):
    N_datasets = len(folder_datasets)
    FrequencyFeatures = []
    FrequencyFeaturesArray = np.zeros((len(folder_datasets),22))
    for i in np.arange(N_datasets):
        dataset = folder_datasets[i]
        spectrum_data = GetSpectrumData(dataset, SamplingRate).with_row_index(name='index', offset=1) 
        N = len(spectrum_data['Sensor1_Magnitude'])
        Rf = TacometerRotatingSpeedEstimate(SamplingRate, N, spectrum_data)
        FrequencyFeatures = [Rf]
        for sensor in spectrum_data.columns[2:9]:
            magnitudes = np.array(GetMagnitudeHarmonics(Rf, spectrum_data, sensor)['HarmonicMagnitude'])
            for magnitude in magnitudes:
                FrequencyFeatures.append(magnitude)
        FrequencyFeaturesArray[i] = FrequencyFeatures

    return FrequencyFeaturesArray

# Extração de Features

## Frequência

In [6]:
ZipFileName = 'dataset/full.zip'
FolderNames = GetSubFolders(ZipFileName)
ColumnNames = ['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'Sensor5', 'Sensor6', 'Sensor7', 'Sensor8']
CastConfig = pl.Float32
SamplingRate = 50000

KeyboardInterrupt: 

In [7]:
# Add Normal datasets
normal_datasets = ReadCsvFromZipFile(ZipFileName, 'normal', ColumnNames, CastConfig)
normal_frequency_features = GetFrequencyFeaturesArray(normal_datasets)


C:\Users\math1\AppData\Local\Temp\ipykernel_15220\633778449.py:21: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))


In [9]:
# Add Horizontal Datasets
horizontal_datasets = ReadCsvFromZipFile(ZipFileName, 'horizontal', ColumnNames, CastConfig)
horizontal_frequency_features = GetFrequencyFeaturesArray(horizontal_datasets)


C:\Users\math1\AppData\Local\Temp\ipykernel_15220\633778449.py:21: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))


In [10]:
# Add Vertical Datasets
vertical_datasets = ReadCsvFromZipFile(ZipFileName, 'vertical', ColumnNames, CastConfig)
vertical_frequency_features =GetFrequencyFeaturesArray(vertical_datasets)


C:\Users\math1\AppData\Local\Temp\ipykernel_15220\633778449.py:21: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))


In [11]:
# Add Imbalance Datasets
imbalance_datasets = ReadCsvFromZipFile(ZipFileName, 'imbalance', ColumnNames, CastConfig)
imbalance_frequency_features =GetFrequencyFeaturesArray(imbalance_datasets)


C:\Users\math1\AppData\Local\Temp\ipykernel_15220\633778449.py:21: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))


In [12]:
# Add Overhang Datasets
overhang_datasets = ReadCsvFromZipFile(ZipFileName, 'overhang', ColumnNames, CastConfig)
overhang_frequency_features =GetFrequencyFeaturesArray(overhang_datasets)

C:\Users\math1\AppData\Local\Temp\ipykernel_15220\633778449.py:21: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))


In [13]:
# Add Underhang Datasets
underhang_datasets =  ReadCsvFromZipFile(ZipFileName, 'underhang', ColumnNames, CastConfig)
underhang_frequency_features =GetFrequencyFeaturesArray(underhang_datasets)

C:\Users\math1\AppData\Local\Temp\ipykernel_15220\633778449.py:21: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  datasets.append(pl.read_csv(zip_file.open(file.filename), has_header=True, new_columns = ColumnNames).cast(CastConfig))


In [14]:
FrequencyFeatures = np.concatenate(
    (normal_frequency_features, 
     horizontal_frequency_features,
     vertical_frequency_features,
     imbalance_frequency_features,
     overhang_frequency_features, 
     underhang_frequency_features), 
     axis=0)
FrequencyFeatures.shape
#np.append(normal_frequency_features, horizontal_frequency_features,vertical_frequency_features,imbalance_frequency_features,overhang_frequency_features, underhang_frequency_features)

(1951, 22)

## Tempo

In [15]:
normal_datasets[0].head()

Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8
f32,f32,f32,f32,f32,f32,f32,f32
4.6038,-0.051295,-0.19405,-0.060071,-0.41809,0.036547,-0.11043,0.11831
4.5703,-0.96908,0.038033,-0.028329,-0.43081,0.041924,-0.14331,-0.071527
4.587,0.89127,0.072973,0.007453,-0.40017,0.04109,-0.11984,0.043445
4.5887,-1.716,-0.32929,-0.033063,-0.50281,0.040474,-0.2527,0.023901
4.5675,1.2403,0.35401,0.04046,-0.36806,0.044062,-0.14258,-0.05488


In [16]:
def Normalization(data):
    data = np.array(data)
    min_data = np.min(data)
    max_data = np.max(data)
    data_norm = (data - min_data) / (max_data - min_data)

    return data_norm



In [17]:
def CalculateEntropy(data):
    raw_hist, raw_bins = np.histogram(data)
    hist = raw_hist[raw_hist != 0] 
    index = np.argwhere(raw_hist == 0)
    bins = np.delete(raw_bins, index)
    P = hist/hist.sum()
    He = -np.sum(P*np.log(P))
    return He

In [18]:
def GetTimeFeatures(rawdataset):
    TimeFeatures = np.zeros((len(rawdataset), 24))
    N_datasets = len(rawdataset)
    for i in np.arange(N_datasets):
        sensor_features = []
        
        # Normalization 
        dataset = Normalization(rawdataset[i])
        dataset = pl.DataFrame(dataset, schema=rawdataset[0].columns)
        
        for sensor in rawdataset[0].columns:
            # Entropy
            He = pl.DataFrame({"He_"+sensor: CalculateEntropy(dataset[sensor])})
            #He = dataset.select(pl.col(sensor).entropy(base=2, normalize=False).alias('He_'+ sensor))
            #He=

            # Mean
            Me = dataset.select(pl.mean(sensor).alias('Me_'+ sensor))

            # Kurtosis
            Ku = dataset.select(pl.col(sensor).kurtosis().alias('Ku_'+ sensor))

            sensor_feature = np.concatenate([He, Me, Ku], axis=1)

            for feature in sensor_feature[0]:
                sensor_features.append(feature)

            #Features = pl.concat([He,Me,Ku], how='horizontal')#.to_numpy()
        TimeFeatures[i] = sensor_features

    return TimeFeatures
    #Features = pl.concat(features, how='horizontal')



In [19]:
# Time Features  - Normal dataset
TimeFeatures_normal = GetTimeFeatures(normal_datasets)
TimeFeatures_normal.shape


(49, 24)

In [20]:
# Time Features  - Horizontal dataset
TimeFeatures_horizontal = GetTimeFeatures(horizontal_datasets)
TimeFeatures_horizontal.shape

(197, 24)

In [21]:
# Time Features  - Vertical dataset
TimeFeatures_vertical = GetTimeFeatures(vertical_datasets)
TimeFeatures_vertical.shape

(301, 24)

In [22]:
# Time Features  - Imbalance dataset
TimeFeatures_imbalance = GetTimeFeatures(imbalance_datasets)
TimeFeatures_imbalance.shape

(333, 24)

In [23]:
# Time Features  - Overhang dataset
TimeFeatures_overhang = GetTimeFeatures(overhang_datasets)
TimeFeatures_overhang.shape

(513, 24)

In [24]:
# Time Features  - Underhang dataset
TimeFeatures_underhang = GetTimeFeatures(underhang_datasets)
TimeFeatures_underhang.shape

(558, 24)

In [25]:
TimeFeatures = np.concatenate(
    (TimeFeatures_normal, 
     TimeFeatures_horizontal,
     TimeFeatures_vertical,
     TimeFeatures_imbalance,
     TimeFeatures_overhang, 
     TimeFeatures_underhang), 
     axis=0)
TimeFeatures.shape

(1951, 24)

# Vetor de Características

In [26]:
Features = np.concatenate((FrequencyFeatures,TimeFeatures), axis=1)
Features.shape

(1951, 46)

In [27]:
# Otimizar esse concat no futuro, pra não ter que fazer na mão.
FeatureColumns = {'Rf': Features[:,0],
'Rf2': Features[:,1],
'2Rf2': Features[:,2],
'3Rf2': Features[:,3],
'Rf3': Features[:,4],
'2Rf3': Features[:,5],
'3Rf3': Features[:,6],
'Rf4': Features[:,7],
'2Rf4': Features[:,8],
'3Rf4': Features[:,9],
'Rf5': Features[:,10],
'2Rf5': Features[:,11],
'3Rf5': Features[:,12],
'Rf6': Features[:,13],
'2Rf6': Features[:,14],
'3Rf6': Features[:,15],
'Rf7': Features[:,16],
'2Rf7': Features[:,17],
'3Rf7': Features[:,18],
'Rf8': Features[:,19],
'2Rf8': Features[:,20],
'3Rf8': Features[:,21],
'He1': Features[:,22],
'Me1': Features[:,23],
'Ku1': Features[:,24],
'He2': Features[:,25],
'Me2': Features[:,26],
'Ku2': Features[:,27],
'He3': Features[:,28],
'Me3': Features[:,29],
'Ku3': Features[:,30],
'He4': Features[:,31],
'Me4': Features[:,32],
'Ku4': Features[:,33],
'He5': Features[:,34],
'Me5': Features[:,35],
'Ku5': Features[:,36],
'He6': Features[:,37],
'Me6': Features[:,38],
'Ku6': Features[:,39],
'He7': Features[:,40],
'Me7': Features[:,41],
'Ku7': Features[:,42],
'He8': Features[:,43],
'Me8': Features[:,44],
'Ku8': Features[:,45]}

FeaturesDataset = pl.DataFrame(FeatureColumns) 
FeaturesDataset.head()

Rf,Rf2,2Rf2,3Rf2,Rf3,2Rf3,3Rf3,Rf4,2Rf4,3Rf4,Rf5,2Rf5,3Rf5,Rf6,2Rf6,3Rf6,Rf7,2Rf7,3Rf7,Rf8,2Rf8,3Rf8,He1,Me1,Ku1,He2,Me2,Ku2,He3,Me3,Ku3,He4,Me4,Ku4,He5,Me5,Ku5,He6,Me6,Ku6,He7,Me7,Ku7,He8,Me8,Ku8
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
24.000192,0.000683,0.000906,0.001137,0.002617,0.001014,0.000445,0.0006,0.000284,0.000356,0.042056,0.005808,0.002233,0.00106,0.0001,0.000172,0.011005,0.002421,0.001005,0.00051,0.001167,0.000078,0.898274,0.381278,4.304769,2.018721,0.380603,-1.06574,1.639317,0.380849,-0.725947,1.64874,0.380965,-0.377444,1.878726,0.384693,-0.387145,1.988329,0.381308,-0.767456,1.750532,0.381528,-0.250984,1.724157,0.381897,0.065311
25.600205,0.002765,0.002409,0.002535,0.004333,0.000661,0.000534,0.000423,0.000272,0.000118,0.05205,0.009884,0.009922,0.001053,0.000139,0.000196,0.013258,0.001201,0.00153,0.000637,0.001072,0.000205,0.928501,0.467355,4.168934,1.962656,0.468465,-0.984737,1.64736,0.466705,-0.660431,1.625401,0.466797,-0.307739,1.919754,0.468241,-0.597151,1.960709,0.467065,-0.766667,1.935416,0.466715,-0.583971,1.661646,0.467566,0.241929
28.000224,0.000602,0.000638,0.000286,0.001239,0.000354,0.001019,0.000129,0.000102,0.000303,0.010226,0.000693,0.005143,0.000433,0.000094,0.000124,0.002414,0.000424,0.000907,0.000201,0.000396,0.000017,0.971581,0.466046,4.138461,1.997051,0.471326,-1.021556,1.682617,0.466381,-0.63432,1.602516,0.466484,-0.226448,1.854107,0.462647,-0.248633,1.939774,0.466688,-0.697357,1.676124,0.465161,-0.018516,1.685009,0.468266,0.063211
29.600237,0.002851,0.001257,0.001715,0.007531,0.002384,0.000071,0.00023,0.000456,0.00011,0.05844,0.007248,0.005113,0.001293,0.002089,0.000203,0.015402,0.002434,0.000283,0.000714,0.001834,0.000097,0.943421,0.469397,3.967974,1.981107,0.472053,-0.99857,1.632274,0.46907,-0.612934,1.659394,0.46918,-0.207611,1.921875,0.465597,-0.34026,1.918103,0.469403,-0.715218,1.713411,0.467762,0.086913,1.687376,0.471156,0.018236
31.600253,0.000253,0.001906,0.000774,0.005902,0.002845,0.000786,0.000117,0.000404,0.000176,0.060609,0.006867,0.002695,0.000802,0.000042,0.000153,0.017627,0.001733,0.000928,0.000655,0.001507,0.000029,0.864854,0.443574,3.814004,1.97154,0.442476,-1.000173,1.679369,0.44298,-0.527783,1.636327,0.44311,-0.159738,1.854548,0.43891,-0.203249,1.984204,0.443367,-0.864867,1.634181,0.436212,0.00973,1.762669,0.444594,-0.100906


In [28]:
FeaturesDataset.shape

(1951, 46)

In [29]:
FeaturesDataset.write_csv('dataset/FeaturesDataset.csv')